In [1]:
import numpy as np
from tqdm import tqdm_notebook as tqdm
import scipy as sp
import scipy.stats as sps
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import gc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import re
import math
import numpy as np
import pandas as pd
import scipy.stats as sps
import matplotlib.pyplot as plt
from numpy.linalg import inv
from numpy import linalg as LA
from scipy.linalg import eigvals as eig
import seaborn as sns
from collections import Counter
from statsmodels.sandbox.stats.multicomp import multipletests
from tqdm import tqdm_notebook
import gc
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_boston
from scipy.linalg import eigvals
sns.set()
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
from collections import defaultdict
from dateutil.parser import parse
import operator

In [2]:
import requests
import pandas as pd
import time
import numpy as np

def execute_query(query, cluster, alias, token, timeout=600):
    proxy = "http://{}.yt.yandex.net".format(cluster)
    s = requests.Session()
    url = "{proxy}/query?database={alias}&password={token}&enable_optimize_predicate_expression=0".format(proxy=proxy, alias=alias, token=token)
    resp = s.post(url, data=query, timeout=timeout)
    resp.raise_for_status()
    rows = resp.text.strip().split('\n')
    return rows

def chyt_execute_query(query, cluster, alias, token, columns):
    i = 0
    while True:
        try:
            result = execute_query(query=query, cluster=cluster, alias=alias, token=token)
            users = pd.DataFrame([row.split('\t') for row in result], columns = columns)
            return users
        except Exception as err:
            print(err)
            i += 1
            if i > 5:
                print('Break Excecution')
                break

cluster = "hahn"
alias = "*cloud_analytics"
token = ''

In [3]:
request = """
SELECT
    puid,
    event, 
    url,
    referer,
    toDateTime(ts) as timestamp
FROM "//home/cloud_analytics/import/console_logs/events"
WHERE 
    response >= '200' 
and
    response < '300'
and 
    puid != ''
AND 
    timestamp > addDays(today(), -30)
ORDER BY ts;
"""

In [4]:
info_request = """
SELECT
    puid,
    max(billing_account_id) as billing_account_id,
    max(user_settings_email) as email,
    max(cloud_id) as cloud_id,
    max(first_first_trial_consumption_datetime) as trial_started,
    max(first_first_paid_consumption_datetime) as paid_started,
    sum(if (service_name == 'mdb', trial_consumption + real_consumption, 0)) as mdb_consumption
FROM "//home/cloud_analytics/cubes/acquisition_cube/cube"
GROUP BY puid
HAVING 
    isNotNull(cloud_id)
AND
    isNotNull(email)
AND
    mdb_consumption < 1;
"""

In [8]:
mails = ['%ClickHouse-Step-by-step%',
        '%MDB-overview%',
        '%MDB-overview-v2%',
        '%MDB-upsell-backup%',
        '%MDB-value%',
        '%Mongo-Step-by-step%',
        '%MySQL-Step-by-step%',
        '%PostgreSQL-Step-by-step%',
        '%Redis-Step-by-step%',
        '%MDB-exp%']

In [9]:
previous_mail_req = f"""
SELECT
    "email",
    "mailing_name"
FROM "//home/cloud_analytics_test/cubes/emailing/cube"
WHERE {"mailing_name like '" + "' or mailing_name like '".join(mails) + "'"}
"""

In [10]:
def make_event_df():
    columns = ["puid", "event", "url", "referer", "datetime"]
    df = chyt_execute_query(query=request, cluster=cluster, 
                       alias=alias, token=token, columns=columns)
    df["event"] = df["event"].str.replace(".com", ".ru")
    df["url"] = df["url"].str.replace(".com", ".ru")
    df["referer"] = df["referer"].str.replace(".com", ".ru")
    df["puid"] = df["puid"].astype(str)
    return df

def make_info_df():
    columns = ["puid", 
               "billing_account_id", 
               "email", "cloud_id",
               "trial_started",
               "paid_started",
               "mdb_consumption"]
    df = chyt_execute_query(query=info_request, cluster=cluster, 
                       alias=alias, token=token, columns=columns)
    df["puid"] = df["puid"].astype(str)
    df.replace("\\N", np.nan, inplace = True)
    return df

def find_previous_mails_df():
    columns = ["mail", 
               "mail_name"]
    df = chyt_execute_query(query=previous_mail_req, cluster=cluster, 
                       alias=alias, token=token, columns=columns)
    df.replace("\\N", np.nan, inplace = True)
    return df


In [11]:
def works_with_emails(mail_):
    mail_parts = str(mail_).split('@')
    if len(mail_parts) > 1:
        if 'yandex.' in mail_parts[1].lower() or 'ya.' in mail_parts[1].lower():
            domain = 'yandex.ru'
            login = mail_parts[0].lower().replace('.', '-')
            return login + '@' + domain
        else:
            return mail_.lower()

In [12]:
databases = set(["postgresql", "mysql", "clickhouse", "mongodb", "redis"])
types = ["docs", "prices", "services"]

In [25]:
def get_mails_with_mailing_before():
    previous_mails = find_previous_mails_df()
    old_emails = previous_mails["mail"].apply(works_with_emails).unique()
    return old_emails

In [26]:
def get_all_dfs():
    old_emails = get_mails_with_mailing_before()
    df = make_event_df()
    info_df = make_info_df()
    return df, info_df, old_emails

In [15]:
def check_database_inside_event(event):
    ans = re.findall(r"|".join(databases), event)
    if ans:
        return ans[0]
    return False

def check_type_inside_event(event):
    ans = re.findall(r"|".join(types), event)
    if ans:
        return ans[0]
    return False

In [16]:
def remove_previous_mails(curr_df_orig, old_emails):
    curr_df = curr_df_orig.copy()
    curr_df["email"] = curr_df["email"].apply(works_with_emails)
    curr_df = curr_df[~curr_df["email"].isin(old_emails)]
    return curr_df

In [17]:
def get_first_mdb_sample(old_emails, df, info_df):
    events = df["event"].unique()
    df_grouped = df.groupby("puid")
    puid_results = {}
    for puid, df_puid in tqdm(df_grouped):
        events = np.array(df_puid["event"])[::-1]
        times = np.array(df_puid["datetime"])[::-1]
        for event, time in zip(events, times):
            if "create-cluster" in event:
                result_database = check_database_inside_event(event)
                if result_database:
                    puid_results[puid] = (puid, result_database, time)
                    break
    first_sample_df = pd.DataFrame.from_dict(puid_results, \
        columns = ["puid", "last_service", "dt_last_vis_create-cluster"], orient = "index")
    final_first_sample = pd.merge(first_sample_df, info_df, on = "puid", how = "inner")
    first_puids = final_first_sample["puid"].unique()
    final_first_sample_df = remove_previous_mails(final_first_sample, old_emails)
    return final_first_sample_df, first_puids

In [18]:
def open_event(dict_of_open_events, event, time):
    dict_of_open_events[event] = time
    return

def close_event_func(dict_of_open_events, event, time, closed_events):
    try:
        db = check_database_inside_event(event)
        closed_events[db] += (time - dict_of_open_events[event]).total_seconds()
        print(event)
        print(dict_of_open_events[event], time)
        del dict_of_open_events[event]
    except Exception:
        pass

In [19]:
def remove_previous_mails(curr_df_orig, old_emails):
    curr_df = curr_df_orig.copy()
    curr_df["email"] = curr_df["email"].apply(works_with_emails)
    curr_df = curr_df[~curr_df["email"].isin(old_emails)]
    return curr_df

In [35]:
def get_second_mdb_sample(old_emails, df, info_df, first_puids):
    events = df["event"].unique()
    df_grouped = df.groupby("puid")
    puid_results = {}
    __SESSION_MAX_TIME = 30 * 60
    resulted_df = {}
    
    for puid, df_puid in tqdm(df_grouped):
        curr_open_events = {}
        visited_databases = defaultdict(set)
        closed_events = Counter()
        prev_time = parse("2018-01-01")

        for _, row in df_puid.iterrows():
            event = row["event"]
            close_event = row["referer"]
            time = parse(row["datetime"])
            if (time - prev_time).total_seconds() > __SESSION_MAX_TIME:
                curr_open_events = {}

            result_database = check_database_inside_event(event)
            typed = check_type_inside_event(event)
            close_database = check_database_inside_event(close_event)
            close_type = check_type_inside_event(close_event)

            if close_database and close_type:
                close_event_func(curr_open_events, close_type + close_database, time, closed_events)

            if result_database and typed:
                visited_databases[result_database].add(typed)
                open_event(curr_open_events, typed + result_database, time)

        candidate = list()
        ans = sorted(visited_databases.items(), key= lambda x: -len(x[1]))
        if not ans:
            continue

        max_len = len(ans[0][1])
        for key, val in ans:
            if len(visited_databases[key]) > 1 and max_len == len(visited_databases[key]):
                candidate.append(key)

        if len(candidate) > 0:
            resulted_df[puid] = (puid, candidate[0], str(visited_databases[candidate[0]]))
            
    second_sample_df = pd.DataFrame.from_dict(resulted_df, \
                    columns = ["puid", "last_service", "visited_pages"], orient = "index")
    final_second_sample = pd.merge(second_sample_df, info_df, on = "puid", how = "inner")
    final_second_sample_without_first_puids = \
    final_second_sample[~final_second_sample["puid"].isin(first_puids)]
    final_second_sample_df = remove_previous_mails(final_second_sample_without_first_puids, old_emails)
    return final_second_sample_df

In [36]:
def add_group(df, number_of_groups = 4):
    df = df.sample(frac=1)
    sz = df.shape[0]
    df["group"] = df.index
    df["group"] = df["group"].apply(lambda x: chr(ord('A') + x % number_of_groups))
    return df

In [37]:
def get_final_samples():
    df, info_df, old_emails = get_all_dfs()
    print("Get info")
    final_first_sample_df, first_puids = get_first_mdb_sample(old_emails, df, info_df)
    print("Get first sample")
    final_second_sample_df = get_second_mdb_sample(old_emails, df, info_df, first_puids)
    print("Get second sample")
    final_first_sample_df = add_group(final_first_sample_df, number_of_groups = 4)
    final_second_sample_df = add_group(final_second_sample_df, number_of_groups = 4)
    final_second_sample_df.to_excel("second_sample_experiment_mdb.xlsx", index = False)
    final_first_sample_df.to_excel("first_sample_experiment_mdb.xlsx", index = False)
    return final_first_sample_df, final_second_sample_df

In [38]:
final_first_sample_df, final_second_sample_df = get_final_samples()

Get info



Get first sample



Get second sample
